<a href="https://colab.research.google.com/github/ArtemMaximov77/Analysis_of_traders_positions/blob/main/Load_new_data_oil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load new data oil

>[Load new data oil](#updateTitle=true&folderId=1GFmu1E8fVIBU67PVByCqoyV0FHqrUa-b&scrollTo=1TjYW0Liak7v)

>>[Загрузка новых данных по WTI](#updateTitle=true&folderId=1GFmu1E8fVIBU67PVByCqoyV0FHqrUa-b&scrollTo=zzfCP1IuZmVn)

>>[Загрузка новых данных по Brent](#updateTitle=true&folderId=1GFmu1E8fVIBU67PVByCqoyV0FHqrUa-b&scrollTo=MDFYsbVheOsy)

>>[Загрузка новых данных с MOEX](#updateTitle=true&folderId=1GFmu1E8fVIBU67PVByCqoyV0FHqrUa-b&scrollTo=xrANMacpn7in)



In [1]:
import pandas as pd

## Загрузка новых данных по WTI

In [2]:
data_wti = pd.read_csv('https://raw.githubusercontent.com/ArtemMaximov77/Analysis_of_traders_positions/main/Disaggregated%20Futures%20Only%20Reports/data_wti.csv')


In [3]:
# Последняя дата в данных
last_date = data_wti['date'].iloc[-1]

In [ ]:
url_zip = 'https://www.cftc.gov/files/dea/history/fut_disagg_xls_2023.zip'
!wget $url_zip
url_unzip = '/content/fut_disagg_xls_2023.zip'
!unzip $url_unzip
df = pd.read_excel('f_year.xls')

In [5]:
df_wti = df[(df['CFTC_Contract_Market_Code'] == '067651') & (df['Report_Date_as_MM_DD_YYYY'] > last_date)]
df_wti = df_wti.drop(df.iloc[:, 23:], axis=1)
df_wti = df_wti.drop(['Market_and_Exchange_Names', 'As_of_Date_In_Form_YYMMDD',
                  'CFTC_Contract_Market_Code', 'CFTC_Market_Code', 'CFTC_Region_Code',
                  'CFTC_Commodity_Code', 'Tot_Rept_Positions_Long_All',
                  'Tot_Rept_Positions_Short_All'], axis=1)
df_wti = df_wti.sort_values('Report_Date_as_MM_DD_YYYY')

In [6]:
columns = ['date', 'oi', 'prod_merc_long', 'prod_merc_short', 'swap_long',
           'swap_short', 'swap_spread', 'm_money_long', 'm_money_short',
           'm_money_spread', 'other_rept_long', 'other_rept_short',
           'other_rept_spread', 'nonrept_long', 'nonrept_short']

df_wti.columns = columns

In [ ]:
url_wti_price = 'https://www.eia.gov/dnav/pet/hist_xls/RWTCd.xls'
!wget $url_wti_price

In [8]:
df_price = pd.read_excel('RWTCd.xls', sheet_name='Data 1', skiprows=[0, 1], names=['date', 'price_wti'])

In [9]:
df_price = df_price[df_price['date'] > last_date]

In [10]:
data_wti_temp = df_wti.merge(df_price, on='date')
data_wti_temp['date'] = data_wti_temp['date'].astype('str')

In [11]:
data_wti = pd.concat([data_wti, data_wti_temp], ignore_index=True)

In [12]:
data_wti.to_csv('data_wti.csv')

## Загрузка новых данных по Brent

In [13]:
data_brent = pd.read_csv('https://raw.githubusercontent.com/ArtemMaximov77/Analysis_of_traders_positions/main/Disaggregated%20Futures%20Only%20Reports/data_brent.csv')

In [ ]:
url = 'https://www.theice.com/publicdocs/futures/COTHist2023.csv'
! wget $url
path = 'COTHist2023.csv'
df = pd.read_csv(path)
df_brent = df[(df['CFTC_Commodity_Code'] == 'B') & (df['FutOnly_or_Combined'] == 'FutOnly')]
df_brent = df_brent.drop(df.iloc[:, 23:], axis=1)
df_brent = df_brent.drop(['Market_and_Exchange_Names', 'As_of_Date_Form_MM/DD/YYYY',
                      'CFTC_Contract_Market_Code', 'CFTC_Market_Code', 'CFTC_Region_Code',
                      'CFTC_Commodity_Code', 'Tot_Rept_Positions_Long_All',
                      'Tot_Rept_Positions_Short_All'], axis=1)

In [15]:
df_brent.columns = columns
df_brent['date'] = pd.to_datetime(df_brent['date'], format='%y%m%d')
df_brent = df_brent[df_brent['date'] > last_date]

In [ ]:
url_brent_price = 'https://www.eia.gov/dnav/pet/hist_xls/RBRTEd.xls'
!wget $url_brent_price

In [17]:
df_price_br = pd.read_excel('RBRTEd.xls', sheet_name='Data 1', skiprows=[0, 1], names=['date', 'price_brent'])

In [18]:
df_price_br = df_price_br[df_price_br['date'] > last_date]

In [19]:
data_br_temp = df_brent.merge(df_price_br, on='date')
data_br_temp['date'] = data_br_temp['date'].astype('str')

In [20]:
data_brent = pd.concat([data_brent, data_br_temp], ignore_index=True)

In [21]:
data_brent.to_csv('data_brent.csv')

## Загрузка новых данных с MOEX

In [22]:
data_brent_moex = pd.read_csv('https://raw.githubusercontent.com/ArtemMaximov77/Analysis_of_traders_positions/main/Disaggregated%20Futures%20Only%20Reports/data_brent_moex.csv')

In [23]:
calendar = data_brent[data_brent['date'] > last_date]['date']
calendar = calendar.astype('str').str.replace('-', '').to_list()

In [ ]:
for day in calendar:
    url = f'https://www.moex.com/ru/derivatives/open-positions-csv.aspx?d={day}&t=1'
    ! wget $url
    path = f'open-positions-csv.aspx?d={day}'
    df = pd.read_csv(path)
    df = df[(df['contract_type'] == 'F') & (df['isin'] == 'BR')]
    if day == calendar[0]:
        df.to_csv('COT_Brent_Moex.csv', mode='a', header=True, index = False)
    else:
        df.to_csv('COT_Brent_Moex.csv', mode='a', header=False, index = False)

In [25]:
df_temp = pd.read_csv('COT_Brent_Moex.csv')

In [26]:
df_brent_moex = pd.DataFrame()
df_brent_moex['date'] = df_temp['moment'].drop_duplicates().reset_index(drop=True)
df_brent_moex['fiz_long'] = df_temp[df_temp['iz_fiz'] == 1]['long_position']\
                          .reset_index(drop=True).astype(int)
df_brent_moex['fiz_short'] = df_temp[df_temp['iz_fiz'] == 1]['short_position']\
                          .reset_index(drop=True).astype(int)
df_brent_moex['yur_long'] = df_temp[df_temp['iz_fiz'] != 1]['long_position']\
                          .reset_index(drop=True).astype(int)
df_brent_moex['yur_short'] = df_temp[df_temp['iz_fiz'] != 1]['short_position']\
                          .reset_index(drop=True).astype(int)
df_brent_moex['oi'] = df_brent_moex['fiz_long'] + df_brent_moex['fiz_short']\
                    + df_brent_moex['yur_long'] + df_brent_moex['yur_short']
df_brent_moex['date'] = pd.to_datetime(df_brent_moex['date'], format='%Y-%m-%d')

df_brent_moex['date'] = df_brent_moex['date'].astype('str')


In [27]:
data_brent_moex = pd.concat([data_brent_moex, df_brent_moex], ignore_index=True)

In [28]:
data_brent_moex.to_csv('data_brent_moex.csv')